# Tutorial 21: Monte Carlo Integration

## PHYS 2600

In [ ]:
# Import cell
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from scipy.special import gamma

## T21.1 - Spheres in higher dimensions

As discussed in lecture, Monte Carlo integration really shines for integrals with a high number of dimensions.  Let's investigate with a simple calculation: finding the volume of a unit sphere (radius $R=1$) in $d$ dimensions.  This will be a good test case because we know the answer exactly:

$$
V_d(1) = \frac{\pi^{d/2}}{\Gamma(d/2+1)}
$$

where $\Gamma$ is the Euler gamma function (available as the `gamma` function from `scipy.special` - I've imported it above.)  For even $d=2k$, the formula simplifies to

$$
V_{2k}(1) = \frac{\pi^k}{k!}
$$

The $d=2$ sphere is just the circle, and we recognize $V_2(1) = \pi$.  The next (probably unfamiliar) example is $d=4$, for which we get $V_4(1) = \pi^2 / 2$.


### Part A

Now let's set the problem up as an integral.  In $d=2$, the equation for a circle is $x^2 + y^2 = 1$.  If we restrict to the first quadrant $x>0, y>0$, we can rewrite this as $y = \sqrt{1-x^2}$ with no ambiguity, and then we have

$$
V_2(1) = 4 \int_0^1 dx\ \sqrt{1-x^2} \approx 4V \langle \sqrt{1 - x^2} \rangle = 4 \langle \sqrt{1-x^2} \rangle = \frac{4}{N} \sum_{i=1}^N \sqrt{1-x_i^2}
$$

where we draw $N$ random points $x_i$, and the 4 is to compensate for restricting to the first quadrant.  Here $V$ is the volume of the sampling region, which is just 1 (the length of the $x$-interval.)

__Follow the comments in the cell below__ to set up and evaluate this integral using `N` random samples.


In [ ]:
N = 1000  # Number of samples

# Draw Ns random numbers in [0,1] with np.random.rand()

# Find E1, the mean of sqrt(1-x^2) over the random numbers x

# Integral V2 is equal to 4 times E1

#

This should give you something reasonably close to the expected answer of $\pi$, but probably slightly off due to random fluctuations. 

### Part B

Actually, it would be better to calculate the __standard error__ as well, so we know how far we _expect_ to be off of the right answer.  We recall the main formula for Monte Carlo integration from lecture,

$$
I \approx V \langle f(\mathbf{x}) \rangle \pm V \sqrt{\frac{\langle f(\mathbf{x})^2 \rangle - \langle f(\mathbf{x}) \rangle^2}{N}} 
$$

For our specific integral, we have

$$
V_2(1) \approx 4 \langle \sqrt{1-x^2} \rangle \pm 4 \sqrt{\frac{\langle 1-x^2 \rangle - \langle \sqrt{1-x^2} \rangle^2}{N}}
$$

In the cell below, __calculate the expectation of $1-x^2$__ and __use it to estimate the standard error__ `err_V2` of your integral from above.  The difference between your result and $\pi$ should fall within the standard error most of the time if you repeat the trial.



In [ ]:
# Find E2 = <1-x^2> using the random draw from above

# Calculate the standard error using E2, E1 and N

#

print(V2, "+/-", err_V2)

### Part C

Now we're ready to deal with the general case.  For any number of dimensions $d$, we can write the volume of a unit sphere as the integral

$$
V_d(1) = 2^d \int_0^1 \int_0^{\sqrt{1-x_1^2}}... \int_0^\sqrt{1-x_1^2-x_2^2-...-x_{d-2}^2} dx_1 dx_2 ... dx_{d-1} \sqrt{1 - \sum_{i=1}^{d-1} x_i^2}
$$

where the $2^d$ comes from picking the higher-dimensional equivalent of the first quadrant.  The limits of integration get a bit tricky in higher dimensions, unfortunately!  Although it's not too bad to deal with, let's start with the conceptually simpler algorithm instead.  Instead of eliminating $x_d$ and getting the square root, we can simply write

$$
V_d(1) = \int_0^1 dx_1 ... dx_d\ \Theta(1 - \sum_{i=1}^{d} x_i^2)
$$
where $\Theta$ is the Heaviside step function.  This takes us back to the "hit-or-miss" Monte Carlo algorithm, since the step function is just 1 for any point inside the unit sphere and 0 outside.

__Implement the function `V_sphere_MC(d, N)` below__, which should compute the volume of the `d`-sphere using `N` samples and the hit-or-miss algorithm:

1. Draw points $(x_1, x_2, ..., x_d)$ randomly over the hypercube ($-1 \leq x_i \leq 1$).
2. Count the number of points $N_{\rm hit}$ inside the unit sphere, i.e. points that satisfy the equation $\sum_{i=1}^d x_i^2 \leq 1$.
3. Return $V_d = V (N_{\rm hit} / N)$, where $V = 2^d$ is the volume of the hypercube we're sampling from.

_(Hint: even though we're dealing with d-dimensional vectors, since we only care about the __lengths__ of the vectors, you don't have to worry about using the dot product; just square all the components of the vector and then add them up.)_

In [ ]:
def V_sphere_MC(d, N):

    # Draw an (N x d) array containing N random vectors of length d
    # The random numbers should be between [-1,1], so rescale them!

    # Square the random array, then sum over the axis of length d,
    # to produce an array of squared vector lengths

    # Use the squared lengths to create a boolean array which is True
    # for any points inside the sphere

    # Use np.count_nonzero on the boolean array,
    # to count N_hit, how many points are in the sphere

    # Vd = (2**d) * N_hit / N

    #

    return Vd


V_sphere_MC(4, 1000)

In [ ]:
# Evaluate in d=2: should be close to pi = 3.1415...
print(V_sphere_MC(2, 10000))

# Evaluate in d=3: should be close to (4/3) pi = 4.18879...
print(V_sphere_MC(3, 10000))

The standard error formula still works for hit-or-miss Monte Carlo!  Since the Heaviside function is equal to its own square, we just have (from the general formula I showed at the start of part B)

$$
\sigma_{d, \rm sem} = \sqrt{\frac{2^d V_d - V_d^2}{N}}
$$

I've provided a _wrapper function_ below that takes your implementation of `V_sphere_MC(d,N)` above and adds an error estimate to it before returning both.

If you try this with $d=2$, how does hit-or-miss compare to the standard error you got in part B?  (Since you're now sampling in one more dimension, you should find that the error in part B was a bit smaller with the same number of samples.)

In [ ]:
def V_sphere_witherr(d, N):
    Vd = V_sphere_MC(d, N)
    sigma_SEM = np.sqrt((2**d * Vd - Vd * Vd) / N)

    return (Vd, sigma_SEM)

In [ ]:
# Testing in d=2 again; should (on most trials) be within 1 sigma_SEM of pi.
# Error is a bit larger for the same number of points, since we're sampling one extra dimension!
V_sphere_witherr(2, 1000)

### Part D

Now that we have a general code working, let's start to turn up the number of dimensions and see how the _curse of dimensionality_ sets in.  __Run the two cells below__, adjusting `d` to try to answer the following questions (__write your answers in the Markdown cell below__):

* Do you get the right answer for d=4 and d=6?
* What qualitative change starts to happen around d=10 with the small numbers of samples?
* How high can you go in `d` before even the final Monte Carlo run with 1 million samples starts to deviate badly from the exact answer?

In [ ]:
d = 4

In [ ]:
def exact_vol(d):
    return np.pi ** (d / 2) / gamma(d / 2 + 1)


print(f"Exact volume: V_{d} = {exact_vol(d)}")

samples = [100, 1000, 10000, 100000, 1000000]

for S in samples:
    mean, err = V_sphere_witherr(d, S)
    print(f"{S} samples: V_{d} = {mean:g} +/- {err:g}")

__The qualitative change you should have noticed around `d=10` or so is due to a simple quirk of higher-dimensional geometry.  We're sampling from a hypercube enclosing the unit sphere, which has volume__

$$
V_{d,\rm{cube}} = 2^d
$$

__We know from 2 and 3 dimensions that the unit sphere fits _inside_ this cube, of course (that's why we can use the cube as a sampling region) - only the corners of the cube are outside the sphere.  But how much space is that?  The ratio of sphere to cube volume is, in even $d$ for simplicity,__

$$
\frac{V_{2k,\rm{sphere}}}{V_{2k,\rm{cube}}} = \frac{\pi^k}{4^k k!}
$$

__This is dropping to zero really, really fast - for $k=5$ ($d=10$) the sphere only takes up 0.2% of the cube, and for $d=20$ the cube's volume is about $10^8$ times larger!  (The difference is still just due to the corners of the cube, but a hypercube has a _lot_ of corners.)__

### Part E

Monte Carlo integration is most often used for high-dimensional integrals.  But another convenient application can be to calculate integrals that are just really messy to set up due to their geometry.  For example, suppose we want to calculate the mass of a steel sphere of radius $R = 1$ m that has a cylindrical hole of radius $r = 0.3$ m bored through the center:

<img src="https://raw.githubusercontent.com/wlough/CU-Phys2600-Fall2025/main/lectures/img/sphere-cyl-hole.png" width=350px />


Assuming a constant density, we just need to know the volume to find the mass.  We can write the equation defining the sphere as:

$$
\begin{cases}
\sqrt{x^2 + y^2 + z^2} &\leq 1, \\
\sqrt{x^2 + y^2} &\geq 0.3.
\end{cases}
$$

This would be annoying to set up by hand, since we have a mixture of spherical and cylindrical symmetry.  But finding the volume using hit-or-miss Monte Carlo is easy - we just generate random points in the cube $-1 \leq x,y,z \leq 1$, and then see if they're in the sphere (and not in the cylindrical hole).

__Modify your hit-and-miss volume Monte Carlo above__ to find the volume of this shape in three dimensions.  (Start by copy-pasting your code from `V_sphere_MC()`, then modify it as needed.  You can hard-code $d=3$ for this problem.)

_(Hint: to decide whether a given point is a "hit" on this shape, you'll almost certainly need to compare it to two masks, one for the sphere and one for the cylindrical hole.  Remember that to combine NumPy masks using logical operations, you need the `np.logical_and()` or `np.logical_or()` functions.)_

In [ ]:
def V_sphere_with_hole_MC(N):
    #

Now run the Monte Carlo and find the volume.  In this case, we have a theoretical estimate for the result:

$$
V = V_{\rm sphere} - V_{\rm cyl} \\
= \frac{4}{3} \pi R^3 - \pi r^2 h \\
= \frac{4}{3} \pi - \pi (0.3)^2 (2) \\
\approx 3.62.
$$

This isn't _quite_ right, because there are a handful of points that are in the cylinder but not in the sphere which are being oversubtracted; you might try using your Monte Carlo simulation to see how much volume that region (in the cylinder, out of the sphere) really is.  But it should be pretty close.

For volume this exercise is sort of trivial, but if the object wasn't constant density, or if we wanted some more complicated property like the moments of inertia, then Monte Carlo would be a nice way to do the integral!  (You'll do a more interesting example on the homework.)

In [ ]:
V_sphere_with_hole_MC(100000)

### Part F _(optional challenge)_

Now, can you go back and use the more complicated integral with the definite limits to implement `V_sphere_MC_v2`, using the standard Monte Carlo integral procedure (in $d-1$ dimensions) instead of hit-or-miss (in $d$ dimensions)?  Here's the formula again:

$$
V_d(1) = 2^d \int_0^1 \int_0^{\sqrt{1-x_1^2}}... \int_0^\sqrt{1-x_1^2-x_2^2-...-x_{d-2}^2} dx_1 dx_2 ... dx_{d-1} \sqrt{1 - \sum_{i=1}^{d-1} x_i^2} \\
= 2^d \int_0^1 dx_1 \int_0^1 dx_2 \Theta(\sqrt{1-x_1^2} - x_2) \int_0^1 dx_3 \Theta(\sqrt{1 - x_1^2 - x_2^2} - x_3) ... \sqrt{1-\sum_{i=1}^{d-1} x_i^2}
$$

(remember, the method as we've formulated it needs a sampling region of known volume, so we again use step functions to get all the $x_i$ out of the limits.  In practice, this means you'll need several masks to only keep random points that are inside the sphere!)

As we saw by comparing to the 2d example, you should get smaller errors for the same number of samples using this form.  We can compute the error without too much trouble, since once again the Heaviside functions square to themselves: if we define

$$
E_2 \equiv 2^d \int_0^1 dx_1 \int_0^1 dx_2 \Theta(\sqrt{1-x_1^2} - x_2) \int_0^1 dx_3 \Theta(\sqrt{1 - x_1^2 - x_2^2} - x_3) ... \left({1-\sum_{i=1}^{d-1} x_i^2}\right)
$$

(i.e. just squaring the square root), then the standard error is

$$
\sigma_d(1) = \sqrt{\frac{2^d E_2 - V_d(1)^2}{N}}
$$

_(Hint: I don't have a hint on this one, this is the hardest optional challenge that I've given all semester!  You can see from the integrals above that this is very messy compared to the simple hit-or-miss algorithm...as is usually the case, __the price of better efficiency is increased complexity.__)_

In [ ]:
def V_sphere_MC_v2(d, N):

    random_points = np.random.rand(d - 1, N)

    #

    return V_mean, V_err

In [ ]:
print(V_sphere_witherr(10, 50000))
print(V_sphere_MC_v2(10, 50000))